In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
def save_model(model_path, save_path, tokenizer_path='./bert_tokenizer/'):
    pool_layer = hub.KerasLayer(model_path, signature="tokens", output_key='pooled_output', trainable=True)
    seq_layer = hub.KerasLayer(model_path, signature="tokens", output_key='sequence_output', trainable=True)
    input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")

    # hack
    for w in pool_layer.weights:
        pool_layer._add_existing_weight(w, trainable=True)
    for w in seq_layer.weights:
        seq_layer._add_existing_weight(w, trainable=True)

    x = input_ids
    type_ids = tf.keras.layers.Lambda(
        lambda x: tf.zeros(shape=tf.shape(x), dtype=tf.int32), name='make_type_ids')(x)
    mask = tf.keras.layers.Lambda(
        lambda x: tf.ones(shape=tf.shape(x), dtype=tf.int32), name='make_mask')(x)

    x_pool = pool_layer({
        'segment_ids': type_ids,
        'input_mask': mask,
        'input_ids': x
    })

    x_seq = seq_layer({
        'segment_ids': type_ids,
        'input_mask': mask,
        'input_ids': x
    })

    pool_model = tf.keras.Model(inputs=input_ids, outputs=x_pool)
    seq_model = tf.keras.Model(inputs=input_ids, outputs=x_seq)

    model = tf.keras.Sequential([
        hub.KerasLayer(tokenizer_path),
        pool_model
    ])
#     model._set_inputs(tf.keras.backend.placeholder((None, None), dtype=tf.int32))
    model._set_inputs(tf.keras.backend.placeholder((None, 1), dtype=tf.string))
    model.save(save_path + '_pool', include_optimizer=False)

    model = tf.keras.Sequential([
        hub.KerasLayer(tokenizer_path),
        seq_model
    ])
#     model._set_inputs(tf.keras.backend.placeholder((None, None), dtype=tf.int32))
    model._set_inputs(tf.keras.backend.placeholder((None, 1), dtype=tf.string))
    model.save(save_path + '_seq', include_optimizer=False)

In [4]:
save_model('./hub/chinese_L-12_H-768_A-12', './output/bert_zh')
save_model('./hub/chinese_wwm_ext_L-12_H-768_A-12', './output/bert_wwm_zh')
save_model('./hub/chinese_roberta_wwm_ext_L-12_H-768_A-12', './output/roberta_wwm_zh')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./output/bert_zh_pool/assets
INFO:tensorflow:Assets written to: ./output/bert_zh_seq/assets
INFO:tensorflow:Assets written to: ./output/bert_wwm_zh_pool/assets
INFO:tensorflow:Assets written to: ./output/bert_wwm_zh_seq/assets
INFO:tensorflow:Assets written to: ./output/roberta_wwm_zh_pool/assets
INFO:tensorflow:Assets written to: ./output/roberta_wwm_zh_seq/assets
